<a href="https://colab.research.google.com/github/Arham-12336/Final_evaluation/blob/main/MetaPath2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git


!pip install pyg-lib
!pip install torch-scatter
!pip install torch-sparse
!pip install torch-geometric

In [ ]:
from torch_geometric.data import HeteroData
import torch
import os
import torch
from torch_geometric.nn import MetaPath2Vec
import pandas as pd
import pickle
import numpy as np
import networkx as nx
import pandas as pd

In [ ]:
pubCent = pd.read_pickle(pubCentPkl)
venCent = pd.read_pickle(venCentPkl)
autCent = pd.read_pickle(autCentPkl)
autCOCI = pd.read_pickle(autCitNet)

In [ ]:
def HetroData():

  data = HeteroData()
  pubCent = pd.read_pickle(pubCentPkl)
  venCent = pd.read_pickle(venCentPkl)
  autCent = pd.read_pickle(autCentPkl)
  autCOCI = pd.read_pickle(autCitNet)
  data['Publication'].x = torch.tensor(pubCent.values)

  data['Venue'].x = torch.tensor(venCent.values)

  data['Author'].x = torch.tensor(autCent.values)

  COCI = pd.read_pickle(citNet)
  DF = pd.read_pickle(metadata).reset_index()
  pubCN = COCI.drop(columns=['Venue_citing','Venue_cited','Title_citing','Title_cited'], axis=1)
  venCN = COCI.drop(columns=['citing','cited','Title_citing','Title_cited'], axis=1)
  autCN = autCOCI.drop(columns=['Venue_citing','Venue_cited','citing','cited'], axis=1)
  autCN = autCN.drop_duplicates()
  autCN.rename(columns = {'autID_citing':'citing','autID_cited':'cited'}, inplace = True)
  pubCent = pubCent.reset_index()
  pubCent['index']=pubCent.index
  pubCent = pubCent.set_index('Publication').drop(columns=['IN','OUT','ACC','Hub','Auth','PR','BC','CC'], axis=1)
  venCent = venCent.reset_index()
  venCent['index']=venCent.index
  venCent = venCent.set_index('Venue').drop(columns=['IN','OUT','ACC','Hub','Auth','PR','BC','CC'], axis=1)
  autCent = autCent.reset_index()
  autCent['index']=autCent.index
  autCent = autCent.set_index('Author').drop(columns=['IN','OUT','ACC','Hub','Auth','PR','BC','CC'], axis=1)
  autCent = autCent.reset_index()
  autCN = autCN.merge(autCent, left_on='citing', right_index=True)
  autCN = autCN.merge(autCent, left_on='cited', right_index=True, suffixes=['_citing','_cited'])
  pubCN = pubCN.merge(pubCent, left_on='citing', right_index=True)
  pubCN = pubCN.merge(pubCent, left_on='cited', right_index=True, suffixes=['_citing','_cited'])
  venCN = venCN.merge(venCent, left_on='Venue_citing', right_index=True)
  venCN = venCN.merge(venCent, left_on='Venue_cited', right_index=True, suffixes=['_citing','_cited'])
  DF_Author = DF.copy()

  DF_Author = DF_Author.merge(pubCent, left_on='DOI', right_index=True)
  DF_Author = DF_Author.merge(autCent, left_on='Author', right_on='Author', suffixes=['_DOI','_Author'])
  DF_Author = DF_Author.drop(columns=['DOI','Title','Venue','Year','Author'],axis=1).groupby('index_DOI').first().reset_index()


  #1st MetaPath
  src = list(DF_Author.index_Author)
  dst = list(DF_Author.index_DOI)
  edge_index = torch.tensor([src, dst])
  data['Author', 'writes', 'Publication'].edge_index = edge_index

  #2nd MetaPath
  src = list(pubCN.index_citing)
  dst = list(pubCN.index_cited)
  edge_index = torch.tensor([src, dst])
  data['Publication', 'cite', 'Publication'].edge_index = edge_index


  #3rd MetaPath
  DF = DF.merge(pubCent, left_on='DOI', right_index=True)
  DF = DF.merge(venCent, left_on='Venue', right_index=True, suffixes=['_DOI','_Venue'])
  DF = DF.drop(columns=['DOI','Title','Venue','Year','Author'],axis=1).groupby('index_DOI').first().reset_index()
  src = list(DF.index_DOI)
  dst = list(DF.index_Venue)
  edge_index = torch.tensor([src, dst])#.t().contiguous()
  data['Publication', 'in', 'Venue'].edge_index = edge_index


  #4th MetaPath
  DF = pd.read_pickle(metadata).reset_index()
  DF = DF.merge(venCent, left_on='Venue', right_index=True)
  DF = DF.merge(autCent, left_on='Author', right_on='Author', suffixes=['_Venue','_Author'])
  DF = DF.drop(columns=['DOI','Title','Venue','Year','Author'],axis=1).groupby('index_Venue').first().reset_index()
  src = list(DF.index_Venue)
  dst = list(DF.index_Author)
  edge_index = torch.tensor([src, dst])#.t().contiguous()
  data['Venue', 'has', 'Author'].edge_index = edge_index

  
  # print(data)
  return data

In [ ]:
Data = HetroData()
Data

HeteroData(
  Publication={ x=[6662, 8] },
  Venue={ x=[2290, 8] },
  Author={ x=[11884, 8] },
  (Author, writes, Publication)={ edge_index=[2, 6219] },
  (Publication, cite, Publication)={ edge_index=[2, 15089] },
  (Publication, in, Venue)={ edge_index=[2, 6354] },
  (Venue, has, Author)={ edge_index=[2, 2265] }
)

In [ ]:

def Metapath(data):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  device = "cpu"

  metapath = [
    ('Author', 'writes', 'Publication'),
    ('Publication', 'cite', 'Publication'),
    ('Publication', 'in', 'Venue'),
    ('Venue', 'has', 'Author')
  ]
  model = MetaPath2Vec(data.edge_index_dict, embedding_dim=128,
                      metapath=metapath, walk_length=50, context_size=3,
                      walks_per_node=3, num_negative_samples=1,
                      sparse=True).to(device)

  loader = model.loader(batch_size=128, shuffle=True, num_workers=3)

  for idx, (pos_rw, neg_rw) in enumerate(loader):
      if idx == 10: break
      print(idx, pos_rw.shape, neg_rw.shape)

  optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

  def train(epoch, log_steps=50, eval_steps=100):
      model.train()

      total_loss = 0
      for i, (pos_rw, neg_rw) in enumerate(loader):
          optimizer.zero_grad()
          loss = model.loss(pos_rw.to(device), neg_rw.to(device))
          loss.backward()
          optimizer.step()

          total_loss += loss.item()
          if (i + 1) % log_steps == 0:
              print((f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                    f'Loss: {total_loss / log_steps:.4f}'))
              total_loss = 0

          if (i + 1) % eval_steps == 0:
              acc = test()
              print((f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                    f'Acc: {acc:.4f}'))
  def test(train_ratio=0.3):
      model.eval()
      z = model('Author', batch=data.y_index_dict['Author'])
      
      y = data.y_dict['Author']

      perm = torch.randperm(z.size(0))
      train_perm = perm[:int(z.size(0) * train_ratio)]
      test_perm = perm[int(z.size(0) * train_ratio):]

      return model.test(z[train_perm], y[train_perm], z[test_perm],
                        y[test_perm], max_iter=1500)
  for epoch in range(1, 6):

    train(epoch)
    print('trian')
   
    print(f'Epoch: {epoch}')

  loaded_model = MetaPath2Vec(data.edge_index_dict, 
                      embedding_dim=128,
                      metapath=metapath,
                      walk_length=5, 
                      context_size=3,
                      walks_per_node=3,
                      num_negative_samples=1,
                      sparse=True
                      ).to(device)

  print("embeddings:",model.embedding.weight)

In [ ]:
Matapath_Result  = Metapath(Data)
Matapath_Result

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


0 torch.Size([18816, 3]) torch.Size([18816, 3])
1 torch.Size([18816, 3]) torch.Size([18816, 3])
2 torch.Size([18816, 3]) torch.Size([18816, 3])
3 torch.Size([18816, 3]) torch.Size([18816, 3])
4 torch.Size([18816, 3]) torch.Size([18816, 3])
5 torch.Size([18816, 3]) torch.Size([18816, 3])
6 torch.Size([18816, 3]) torch.Size([18816, 3])
7 torch.Size([18816, 3]) torch.Size([18816, 3])
8 torch.Size([18816, 3]) torch.Size([18816, 3])
9 torch.Size([18816, 3]) torch.Size([18816, 3])
Epoch: 1, Step: 00050/93, Loss: 5.6696
trian
Epoch: 1
Epoch: 2, Step: 00050/93, Loss: 4.4610
trian
Epoch: 2
Epoch: 3, Step: 00050/93, Loss: 3.5327
trian
Epoch: 3
Epoch: 4, Step: 00050/93, Loss: 2.7133
trian
Epoch: 4
Epoch: 5, Step: 00050/93, Loss: 2.0012
trian
Epoch: 5
embeddings: Parameter containing:
tensor([[-0.7514, -0.9259,  1.0463,  ...,  0.2749,  0.9522,  0.8225],
        [-1.2731,  0.2759, -0.4865,  ...,  0.3546,  1.8139, -0.6559],
        [-0.6534, -0.2968,  0.8695,  ..., -0.8537,  0.3917, -0.1902],
      